In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import re

In [3]:
# Load the dataset
data_path = '/Users/kalyani/Desktop/AIP/data_cleaning/women_clean_data.csv'  # Update this path
df = pd.read_csv(data_path)

In [4]:
# Check data types
print("Data Types:")
print(df.dtypes)

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

Data Types:
White          object
Black          object
Result         object
TimeControl    object
Moves          object
dtype: object

Missing Values:
White            0
Black            0
Result           0
TimeControl    137
Moves           71
dtype: int64


In [5]:
# Ensure Moves are strings and handle any NaNs
df['Moves'] = df['Moves'].astype(str).replace('nan', '')

In [6]:
# Use Pandas for splitting moves more efficiently and validate chess moves
df['ValidMoves'] = df['Moves'].apply(lambda x: [move for move in x.split() if re.match(r'^[a-h][1-8][qrbn]?$|^[KQRBNA]?x?[a-h][1-8]|O-O(-O)?$', move)])

In [7]:
# Filter out rows where there are fewer than 2 valid moves
df = df[df['ValidMoves'].apply(len) > 1]

# Create the next_moves DataFrame
next_moves = []
for index, row in df.iterrows():
    valid_moves = row['ValidMoves']
    for i in range(len(valid_moves) - 1):
        next_moves.append({
            'White': row['White'],
            'Black': row['Black'],
            'Result': row['Result'],
            'TimeControl': row['TimeControl'],
            'CurrentMove': valid_moves[i],
            'NextMove': valid_moves[i + 1]
        })

next_moves_df = pd.DataFrame(next_moves)

In [8]:
next_moves_df

,White,Black,Result,TimeControl,CurrentMove,NextMove
0,"Goryachkina, Aleksandra","Koutlas, Nikolaos",1-0,180+2,e4,c5
1,"Goryachkina, Aleksandra","Koutlas, Nikolaos",1-0,180+2,c5,Nf3
2,"Goryachkina, Aleksandra","Koutlas, Nikolaos",1-0,180+2,Nf3,d6
3,"Goryachkina, Aleksandra","Koutlas, Nikolaos",1-0,180+2,d6,Bb5+
4,"Goryachkina, Aleksandra","Koutlas, Nikolaos",1-0,180+2,Bb5+,Nd7
...,...,...,...,...,...,...
1014066,"Shuvalova, Polina",Necromon,0-1,180+2,Re1,Kd7
1014067,"Shuvalova, Polina",Necromon,0-1,180+2,Kd7,Re6
1014068,"Shuvalova, Polina",Necromon,0-1,180+2,Re6,Rh7
1014069,"Shuvalova, Polina",Necromon,0-1,180+2,Rh7,Rf6


In [9]:
# Check the new DataFrame
print("\nNext Moves DataFrame:")
print(next_moves_df.head())


Next Moves DataFrame:
                     White              Black Result TimeControl CurrentMove  \
0  Goryachkina, Aleksandra  Koutlas, Nikolaos    1-0       180+2          e4   
1  Goryachkina, Aleksandra  Koutlas, Nikolaos    1-0       180+2          c5   
2  Goryachkina, Aleksandra  Koutlas, Nikolaos    1-0       180+2         Nf3   
3  Goryachkina, Aleksandra  Koutlas, Nikolaos    1-0       180+2          d6   
4  Goryachkina, Aleksandra  Koutlas, Nikolaos    1-0       180+2        Bb5+   

  NextMove  
0       c5  
1      Nf3  
2       d6  
3     Bb5+  
4      Nd7  


In [10]:
# Encode categorical variables
label_encoders = {}
for column in ['White', 'Black', 'Result', 'TimeControl', 'CurrentMove', 'NextMove']:
    le = LabelEncoder()
    next_moves_df[column] = le.fit_transform(next_moves_df[column])
    label_encoders[column] = le  # Store label encoder for future use

In [11]:
# Optimize data types to reduce memory usage
next_moves_df['White'] = next_moves_df['White'].astype('int32')
next_moves_df['Black'] = next_moves_df['Black'].astype('int32')
next_moves_df['Result'] = next_moves_df['Result'].astype('int32')
next_moves_df['TimeControl'] = next_moves_df['TimeControl'].astype('int32')
next_moves_df['CurrentMove'] = next_moves_df['CurrentMove'].astype('int32')
next_moves_df['NextMove'] = next_moves_df['NextMove'].astype('int32')

# Prepare features and target variable
X = next_moves_df[['White', 'Black', 'Result', 'TimeControl', 'CurrentMove']]
y = next_moves_df['NextMove']

In [12]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Train the Random Forest model with reduced number of estimators and max depth to prevent memory issues
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, n_jobs=1, random_state=42, verbose=2)
rf_model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   35.5s finished


RandomForestClassifier(max_depth=10, n_estimators=50, n_jobs=1, random_state=42,
                       verbose=2)

In [14]:
# Check the accuracy
accuracy = rf_model.score(X_test, y_test)
print("\nModel Accuracy:", accuracy)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  2.1min finished



Model Accuracy: 0.046638562236520964


In [17]:
# Make predictions
y_pred = rf_model.predict(X_test)

# Display the predictions
print("\nPredictions on the test set:")
print(y_pred)

# If you want to see the actual vs predicted values
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print("\nActual vs Predicted:")
print(results_df.head())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  2.2min finished



Predictions on the test set:
[ 361 1427  719 ... 1555  719  351]

Actual vs Predicted:
        Actual  Predicted
573788     352        361
634281     498       1427
547581    1003        719
533130      58        719
283739     201        719
